In [ ]:
import os
import shutil
from PIL import Image 
from tensorflow import keras
import numpy as np
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)


def ziffer_data_files(input_dir):
    imgfiles = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if (file.endswith(".jpg") and not file.startswith("10_") and not file.startswith("N")):
                #print(root + "/" + file)
                imgfiles.append(root + "/" + file)
    return  imgfiles

def ziffer_data(input_dir='images'):
    
    files = ziffer_data_files(input_dir)
    
    y_data = np.empty((len(files)))
    y_file = np.empty((len(files)), dtype="<U250")
    x_data = np.empty((len(files),32,20,3))

    for i, aktfile in enumerate(files):
        base = os.path.basename(aktfile)
        # get label from filename (1.2_ new or 1_ old),
        if (base[1]=="."):
            target = base[0:3]
        else:
            target = base[0:1]
         
        category = float(target)
        
        test_image = Image.open(aktfile).resize((20, 32))
        test_image = np.array(test_image, dtype="float32")
        y_file[i] =  str(base)
        x_data[i] = test_image
        y_data[i] =  category
    print("meter digit count: ", len(y_data))   
    return x_data, y_data.reshape(-1), y_file.reshape(-1)
x_md, y_md, f_md = ziffer_data(os.environ['GITHUB_WORKSPACE']+'/images')

logger.debug(f_md.dtype)

In [ ]:
import os
from PIL import Image
import shutil

output_dir = os.environ['GITHUB_WORKSPACE']+'/datasets/meterdigits'
shutil.rmtree(output_dir, ignore_errors=True)
os.makedirs(output_dir, exist_ok=True)


for x, y, f in zip(x_md, y_md, f_md):
    image = Image.fromarray(x.astype(np.uint8), mode='RGB')
    os.makedirs(os.path.join(output_dir, str(y)), exist_ok=True)
    image.save(os.path.join(output_dir, str(y), str(f)))

In [ ]:
import tensorflow as tf
ds = tf.keras.utils.image_dataset_from_directory(directory=output_dir, 
                                            image_size=(32,28), 
                                            batch_size=32,
                                            color_mode='rgb',
                                            label_mode='categorical',
                                            shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

def plot_dataset_it(data_iter, columns=9, rows=5):

    fig = plt.figure(figsize=(18, 11))
    
    for i in range(1, columns*rows +1):
        img, label = data_iter.next()
        fig.add_subplot(rows, columns, i)
        plt.xticks([0.2, 0.4, 0.6, 0.8])
        plt.title(np.argmax(label)/10)  # set title
        plt.imshow(np.array(img[0]).astype(np.uint8), aspect='1.6', extent=[0, 1, 0, 1])
        ax=plt.gca()
        ax.get_xaxis().set_visible(False) 
        # yellow lines
        for y in np.arange(0.2, 0.8, 0.2):
                plt.axhline(y=y,color='yellow')
    plt.show()

plot_dataset_it(iter(ds))